In [3]:
from pyamaze import maze, agent, textLabel

# Create a maze with given dimensions
m = maze(5, 5)

# Generate the maze
m.CreateMaze()

# Optional: Add an agent to the maze
a = agent(m, footprints=True)

# Optional: Add labels to the maze
l = textLabel(m, 'Maze', '5x5 Maze')

# Run the maze
m.run()

In [ ]:
#dfs 
from pyamaze import maze, agent, textLabel

class Node:
    def __init__(self, x, y, parent=None):
        self.x = x
        self.y = y
        self.parent = parent

def dfs(m, start, goal):
    stack = [Node(start[0], start[1])]
    visited = set()
    while stack:
        current = stack.pop()
        if (current.x, current.y) == goal:
            # Reconstruct the path from start to goal
            path = {}
            while current.parent is not None:
                path[(current.parent.x, current.parent.y)] = (current.x, current.y)
                current = current.parent
            return path
        if (current.x, current.y) not in visited:
            visited.add((current.x, current.y))
            for direction in 'ESNW':
                if m.maze_map[current.x, current.y][direction]:
                    x, y = current.x, current.y
                    if direction == 'E':
                        child = Node(x, y+1, current)
                    elif direction == 'W':
                        child = Node(x, y-1, current)
                    elif direction == 'S':
                        child = Node(x+1, y, current)
                    elif direction == 'N':
                        child = Node(x-1, y, current)
                    stack.append(child)
    return None

# Create the maze using pyamaze
m = maze(10, 10)
m.CreateMaze()

# Perform DFS on the maze
start_pos = (m.rows, m.cols)
goal_pos = (1, 1)
path = dfs(m, start_pos, goal_pos)

# If a path is found, trace it with an agent
if path:
    a = agent(m, footprints=True)
    m.tracePath({a:path})
    l = textLabel(m, 'DFS Path', 'Depth-First Search')
    m.run()
else:
    print('No path found from start to end.')


In [ ]:
#bfs
from pyamaze import maze, agent, textLabel
from collections import deque

class Node:
    def __init__(self, x, y, parent=None):
        self.x = x
        self.y = y
        self.parent = parent

def bfs(m, start, goal):
    queue = deque([Node(start[0], start[1])])
    visited = set()
    while queue:
        current = queue.popleft()
        if (current.x, current.y) == goal:
            # Reconstruct the path from start to goal
            path = {}
            while current.parent is not None:
                path[(current.parent.x, current.parent.y)] = (current.x, current.y)
                current = current.parent
            return path
        if (current.x, current.y) not in visited:
            visited.add((current.x, current.y))
            for direction in 'ESNW':
                if m.maze_map[current.x, current.y][direction]:
                    x, y = current.x, current.y
                    if direction == 'E':
                        child = Node(x, y+1, current)
                    elif direction == 'W':
                        child = Node(x, y-1, current)
                    elif direction == 'S':
                        child = Node(x+1, y, current)
                    elif direction == 'N':
                        child = Node(x-1, y, current)
                    if (child.x, child.y) not in visited:
                        queue.append(child)
    return None

# Create the maze using pyamaze
m = maze(10, 10)
m.CreateMaze()

# Perform BFS on the maze
start_pos = (m.rows, m.cols)
goal_pos = (1, 1)
bfs_path = bfs(m, start_pos, goal_pos)

# If a BFS path is found, trace it with an agent
if bfs_path:
    b = agent(m, footprints=True, color='cyan')
    m.tracePath({b:bfs_path})
    l = textLabel(m, 'BFS Path', 'Breadth-First Search')
    m.run()
else:
    print('No path found from start to end using BFS.')


In [ ]:
#A*
from pyamaze import maze, agent, textLabel
from queue import PriorityQueue

class Node:
    def __init__(self, x, y, parent=None):
        self.x = x
        self.y = y
        self.parent = parent
        self.g = 0 # Cost from start to current Node
        self.h = 0 # Heuristic based estimated cost from current to end
        self.f = 0 # Total cost

    def __eq__(self, other):
        return self.x == other.x and self.y == other.y

    def __lt__(self, other):
        return self.f < other.f

def heuristic(a, b):
    # Use Manhattan distance as heuristic
    return abs(a.x - b.x) + abs(a.y - b.y)

def a_star(m, start, end):
    # Initialize both open and closed list
    open_list = PriorityQueue()
    open_list.put((0, start))
    # Initialize whole grid
    grid = [[Node(x, y) for y in range(1, m.cols+1)] for x in range(1, m.rows+1)]
    grid[start.x-1][start.y-1].g = grid[start.x-1][start.y-1].h = grid[start.x-1][start.y-1].f = 0

    while not open_list.empty():
        _, current_node = open_list.get()
        # Found the goal
        if (current_node.x, current_node.y) == (end.x, end.y):
            path = {}
            while current_node.parent is not None:
                path[(current_node.parent.x, current_node.parent.y)] = (current_node.x, current_node.y)
                current_node = current_node.parent
            return path

        for direction in 'ESNW':
            if m.maze_map[current_node.x, current_node.y][direction]:
                if direction == 'E':
                    child = Node(current_node.x, current_node.y+1, current_node)
                elif direction == 'W':
                    child = Node(current_node.x, current_node.y-1, current_node)
                elif direction == 'S':
                    child = Node(current_node.x+1, current_node.y, current_node)
                elif direction == 'N':
                    child = Node(current_node.x-1, current_node.y, current_node)

                child.g = current_node.g + 1
                child.h = heuristic(child, end)
                child.f = child.g + child.h

                # Child is already in the open list
                for _, open_node in open_list.queue:
                    if child == open_node and child.g > open_node.g:
                        break
                else:
                    open_list.put((child.f, child))
    return None

# Create a 10x10 maze using pyamaze
m = maze(10, 10)
m.CreateMaze()

start = Node(m.rows, m.cols)
end = Node(1, 1)
a_star_path = a_star(m, start, end)

# If an A* path is found, trace it with an agent
if a_star_path:
    a = agent(m, x=start.x, y=start.y, footprints=True, color='yellow')
    m.tracePath({a:a_star_path})
    l = textLabel(m, 'A* Path', 'A* Algorithm')
    m.run()
else:
    print('No path found from start to end using A*.')


In [1]:
from pyamaze import maze, agent, textLabel
from collections import deque
from queue import PriorityQueue

class Node:
    def __init__(self, x, y, parent=None):
        self.x = x
        self.y = y
        self.parent = parent
        self.g = 0
        self.h = 0
        self.f = 0

    def __eq__(self, other):
        return self.x == other.x and self.y == other.y

    def __lt__(self, other):
        return self.f < other.f

def heuristic(a, b):
    return abs(a.x - b.x) + abs(a.y - b.y)

def dfs(m, start, goal):
    stack = [Node(start[0], start[1])]
    visited = set()
    while stack:
        current = stack.pop()
        if (current.x, current.y) == goal:
            # Reconstruct the path from start to goal
            path = {}
            while current.parent is not None:
                path[(current.parent.x, current.parent.y)] = (current.x, current.y)
                current = current.parent
            return path
        if (current.x, current.y) not in visited:
            visited.add((current.x, current.y))
            for direction in 'ESNW':
                if m.maze_map[current.x, current.y][direction]:
                    x, y = current.x, current.y
                    if direction == 'E':
                        child = Node(x, y+1, current)
                    elif direction == 'W':
                        child = Node(x, y-1, current)
                    elif direction == 'S':
                        child = Node(x+1, y, current)
                    elif direction == 'N':
                        child = Node(x-1, y, current)
                    stack.append(child)
    return None
    
    

def bfs(m, start, goal):
    queue = deque([Node(start[0], start[1])])
    visited = set()
    while queue:
        current = queue.popleft()
        if (current.x, current.y) == goal:
            # Reconstruct the path from start to goal
            path = {}
            while current.parent is not None:
                path[(current.parent.x, current.parent.y)] = (current.x, current.y)
                current = current.parent
            return path
        if (current.x, current.y) not in visited:
            visited.add((current.x, current.y))
            for direction in 'ESNW':
                if m.maze_map[current.x, current.y][direction]:
                    x, y = current.x, current.y
                    if direction == 'E':
                        child = Node(x, y+1, current)
                    elif direction == 'W':
                        child = Node(x, y-1, current)
                    elif direction == 'S':
                        child = Node(x+1, y, current)
                    elif direction == 'N':
                        child = Node(x-1, y, current)
                    if (child.x, child.y) not in visited:
                        queue.append(child)
    return None
    
    
    
def a_star(m, start, goal):
    open_list = PriorityQueue()
    start_node = Node(start.x, start.y)
    goal_node = Node(goal.x, goal.y)
    start_node.g = start_node.h = start_node.f = 0
    goal_node.g = goal_node.h = goal_node.f = 0
    open_list.put((start_node.f, start_node))

    # Initialize whole grid
    grid = [[Node(x, y) for y in range(1, m.cols+1)] for x in range(1, m.rows+1)]
    grid[start.x-1][start.y-1].g = grid[start.x-1][start.y-1].h = grid[start.x-1][start.y-1].f = 0

    while not open_list.empty():
        _, current_node = open_list.get()
        # Found the goal
        if (current_node.x, current_node.y) == (end.x, end.y):
            path = {}
            while current_node.parent is not None:
                path[(current_node.parent.x, current_node.parent.y)] = (current_node.x, current_node.y)
                current_node = current_node.parent
            return path

        for direction in 'ESNW':
            if m.maze_map[current_node.x, current_node.y][direction]:
                if direction == 'E':
                    child = Node(current_node.x, current_node.y+1, current_node)
                elif direction == 'W':
                    child = Node(current_node.x, current_node.y-1, current_node)
                elif direction == 'S':
                    child = Node(current_node.x+1, current_node.y, current_node)
                elif direction == 'N':
                    child = Node(current_node.x-1, current_node.y, current_node)

                child.g = current_node.g + 1
                child.h = heuristic(child, end)
                child.f = child.g + child.h

                # Child is already in the open list
                for _, open_node in open_list.queue:
                    if child == open_node and child.g > open_node.g:
                        break
                else:
                    open_list.put((child.f, child))
    return None
    
    
    

m = maze(20, 20)
m.CreateMaze(loopPercent=100)

# Coordinates for start and end positions (as tuples)
start_coords = (m.rows, m.cols)
end_coords = (1, 1)

# Ask user which algorithm to use
print('Choose an algorithm:')
print('1: DFS')
print('2: BFS')
print('3: A*')
choice = input('Enter your choice (1, 2, 3): ')

# Convert start and end positions to Node objects for A*
start_node = Node(start_coords[0], start_coords[1])
end_node = Node(end_coords[0], end_coords[1])

path = None
algo_text = ''

if choice == '1':
    print('Running DFS...')
    path = dfs(m, start_coords, end_coords)  # DFS expects tuples
    algo_text = 'DFS'
elif choice == '2':
    print('Running BFS...')
    path = bfs(m, start_coords, end_coords)  # BFS expects tuples
    algo_text = 'BFS'
elif choice == '3':
    print('Running A*...')
    path = a_star(m, start_node, end_node)  # A* expects Node objects
    algo_text = 'A*'
else:
    print('Invalid choice.')
    exit()

# If a path is found, trace it with an agent
if path:
    a = agent(m, x=start_coords[0], y=start_coords[1], footprints=True)
    # Convert the path to the format expected by pyamaze:
    pyamaze_path = {(n.x, n.y): (n.parent.x, n.parent.y) for n in path.values() if n.parent is not None}
    m.tracePath({a: pyamaze_path})
    l = textLabel(m, f'{algo_text} Path Length', len(pyamaze_path))
    m.run()
else:
    print(f'No path found from start to end using {algo_text}.')

Choose an algorithm:
1: DFS
2: BFS
3: A*
Enter your choice (1, 2, 3): 2
Running BFS...


AttributeError: 'tuple' object has no attribute 'parent'

In [2]:
from pyamaze import maze, agent, textLabel
from collections import deque
from queue import PriorityQueue

class Node:
    def __init__(self, x, y, parent=None):
        self.x = x
        self.y = y
        self.parent = parent
        self.g = 0
        self.h = 0
        self.f = 0

    def __eq__(self, other):
        return self.x == other.x and self.y == other.y

    def __lt__(self, other):
        return self.f < other.f

def heuristic(a, b):
    return abs(a.x - b.x) + abs(a.y - b.y)

def dfs(m, start, goal):
    stack = [Node(start[0], start[1])]
    visited = set()
    path = []
    dead_ends = []

    while stack:
        current = stack.pop()
        current_coords = (current.x, current.y)
        if current_coords == goal:
            # Reconstruct the path from start to goal
            while current is not None:
                path.append(current_coords)
                current = current.parent
                if current is not None:
                    current_coords = (current.x, current.y)
            return path, dead_ends

        if current_coords not in visited:
            visited.add(current_coords)
            for direction in 'ESNW':
                if m.maze_map[current.x, current.y][direction]:
                    x, y = current.x, current.y
                    if direction == 'E':
                        child_coords = (x, y+1)
                    elif direction == 'W':
                        child_coords = (x, y-1)
                    elif direction == 'S':
                        child_coords = (x+1, y)
                    elif direction == 'N':
                        child_coords = (x-1, y)
                    if child_coords not in visited:
                        stack.append(Node(*child_coords, current))
                else:
                    dead_ends.append(current_coords)

    return None, dead_ends


    
    

def bfs(m, start, goal):
    queue = deque([Node(start[0], start[1])])
    visited = set()
    while queue:
        current = queue.popleft()
        if (current.x, current.y) == goal:
            # Reconstruct the path from start to goal
            path = {}
            while current.parent is not None:
                path[(current.parent.x, current.parent.y)] = (current.x, current.y)
                current = current.parent
            return path
        if (current.x, current.y) not in visited:
            visited.add((current.x, current.y))
            for direction in 'ESNW':
                if m.maze_map[current.x, current.y][direction]:
                    x, y = current.x, current.y
                    if direction == 'E':
                        child = Node(x, y+1, current)
                    elif direction == 'W':
                        child = Node(x, y-1, current)
                    elif direction == 'S':
                        child = Node(x+1, y, current)
                    elif direction == 'N':
                        child = Node(x-1, y, current)
                    if (child.x, child.y) not in visited:
                        queue.append(child)
    return None
    
    
    
def a_star(m, start, goal):
     # Initialize both open and closed list
    open_list = PriorityQueue()
    open_list.put((0, start))
    # Initialize whole grid
    grid = [[Node(x, y) for y in range(1, m.cols+1)] for x in range(1, m.rows+1)]
    grid[start.x-1][start.y-1].g = grid[start.x-1][start.y-1].h = grid[start.x-1][start.y-1].f = 0

    while not open_list.empty():
        _, current_node = open_list.get()
        # Found the goal
        if (current_node.x, current_node.y) == (end.x, end.y):
            path = {}
            while current_node.parent is not None:
                path[(current_node.parent.x, current_node.parent.y)] = (current_node.x, current_node.y)
                current_node = current_node.parent
            return path

        for direction in 'ESNW':
            if m.maze_map[current_node.x, current_node.y][direction]:
                if direction == 'E':
                    child = Node(current_node.x, current_node.y+1, current_node)
                elif direction == 'W':
                    child = Node(current_node.x, current_node.y-1, current_node)
                elif direction == 'S':
                    child = Node(current_node.x+1, current_node.y, current_node)
                elif direction == 'N':
                    child = Node(current_node.x-1, current_node.y, current_node)

                child.g = current_node.g + 1
                child.h = heuristic(child, end)
                child.f = child.g + child.h

                # Child is already in the open list
                for _, open_node in open_list.queue:
                    if child == open_node and child.g > open_node.g:
                        break
                else:
                    open_list.put((child.f, child))
    return None
    
    
    

# Create a 20x20 maze using pyamaze
m = maze(5, 5)
m.CreateMaze(loopPercent=100)

# Define start and end coordinates
start_coords = (m.rows, m.cols)  # Start position (usually bottom-right corner)
end_coords = (1, 1)              # End position (usually top-left corner)

# Ask user which algorithm to use
print('Choose an algorithm:')
print('1: DFS')
print('2: BFS')
print('3: A*')
choice = input('Enter your choice (1, 2, 3): ')
path = None
algo_text = ''
# Run the chosen algorithm
if choice == '1':
    print('Running DFS...')
    final_path, dead_ends = dfs(m, start_coords, end_coords)
    if final_path:
        pyamaze_path = {(n[0], n[1]): 'blue' for n in final_path}
        for n in dead_ends:
            pyamaze_path[(n[0], n[1])] = 'red'
        # Visualize the path in pyamaze
        # Note: You need to implement a way to visualize the path in pyamaze
    else:
        print('No path found using DFS')

elif choice == '2':
    print('Running BFS...')
    path = bfs(m, start, end)
    algo_text = 'BFS'
elif choice == '3':
    print('Running A*...')
    path = a_star(m, start, end)
    algo_text = 'A*'
else:
    print('Invalid choice.')

# If a path is found, trace it with an agent
if path:
    a = agent(m, x=start[0], y=start[1], footprints=True)
    m.tracePath({a:path})
    l = textLabel(m, f'{algo_text} Path', f'{algo_text} Algorithm')
    m.run()
else:
    print(f'No path found from start to end using {algo_text}.')

Choose an algorithm:
1: DFS
2: BFS
3: A*
Enter your choice (1, 2, 3): 2
Running BFS...


NameError: name 'start' is not defined

In [ ]:
from pyamaze import maze, agent, textLabel
from collections import deque
from queue import PriorityQueue

class Node:
    def __init__(self, x, y, parent=None):
        self.x = x
        self.y = y
        self.parent = parent
        self.g = 0
        self.h = 0
        self.f = 0

    def __eq__(self, other):
        return self.x == other.x and self.y == other.y

    def __lt__(self, other):
        return self.f < other.f

def heuristic(a, b):
    return abs(a.x - b.x) + abs(a.y - b.y)

def dfs(m, start, goal):
    stack = [Node(start[0], start[1])]
    visited = set()
    while stack:
        current = stack.pop()
        if (current.x, current.y) == goal:
            # Reconstruct the path from start to goal
            path = {}
            while current.parent is not None:
                path[(current.parent.x, current.parent.y)] = (current.x, current.y)
                current = current.parent
            return path
        if (current.x, current.y) not in visited:
            visited.add((current.x, current.y))
            for direction in 'ESNW':
                if m.maze_map[current.x, current.y][direction]:
                    x, y = current.x, current.y
                    if direction == 'E':
                        child = Node(x, y+1, current)
                    elif direction == 'W':
                        child = Node(x, y-1, current)
                    elif direction == 'S':
                        child = Node(x+1, y, current)
                    elif direction == 'N':
                        child = Node(x-1, y, current)
                    stack.append(child)
    return None
    
    

def bfs(m, start, goal):
    queue = deque([Node(start[0], start[1])])
    visited = set()
    while queue:
        current = queue.popleft()
        if (current.x, current.y) == goal:
            # Reconstruct the path from start to goal
            path = {}
            while current.parent is not None:
                path[(current.parent.x, current.parent.y)] = (current.x, current.y)
                current = current.parent
            return path
        if (current.x, current.y) not in visited:
            visited.add((current.x, current.y))
            for direction in 'ESNW':
                if m.maze_map[current.x, current.y][direction]:
                    x, y = current.x, current.y
                    if direction == 'E':
                        child = Node(x, y+1, current)
                    elif direction == 'W':
                        child = Node(x, y-1, current)
                    elif direction == 'S':
                        child = Node(x+1, y, current)
                    elif direction == 'N':
                        child = Node(x-1, y, current)
                    if (child.x, child.y) not in visited:
                        queue.append(child)
    return None
    
    
    
def a_star(m, start, goal):
     # Initialize both open and closed list
    open_list = PriorityQueue()
    open_list.put((0, start))
    # Initialize whole grid
    grid = [[Node(x, y) for y in range(1, m.cols+1)] for x in range(1, m.rows+1)]
    grid[start.x-1][start.y-1].g = grid[start.x-1][start.y-1].h = grid[start.x-1][start.y-1].f = 0

    while not open_list.empty():
        _, current_node = open_list.get()
        # Found the goal
        if (current_node.x, current_node.y) == (end.x, end.y):
            path = {}
            while current_node.parent is not None:
                path[(current_node.parent.x, current_node.parent.y)] = (current_node.x, current_node.y)
                current_node = current_node.parent
            return path

        for direction in 'ESNW':
            if m.maze_map[current_node.x, current_node.y][direction]:
                if direction == 'E':
                    child = Node(current_node.x, current_node.y+1, current_node)
                elif direction == 'W':
                    child = Node(current_node.x, current_node.y-1, current_node)
                elif direction == 'S':
                    child = Node(current_node.x+1, current_node.y, current_node)
                elif direction == 'N':
                    child = Node(current_node.x-1, current_node.y, current_node)

                child.g = current_node.g + 1
                child.h = heuristic(child, end)
                child.f = child.g + child.h

                # Child is already in the open list
                for _, open_node in open_list.queue:
                    if child == open_node and child.g > open_node.g:
                        break
                else:
                    open_list.put((child.f, child))
    return None
    
    
    

# Create a 20x20 maze using pyamaze
m = maze(20, 20)
m.CreateMaze(loopPercent=100)

# Coordinates for start and end positions
start = (m.rows, m.cols)
end = (1, 1)

# Ask user which algorithm to use
print('Choose an algorithm:')
print('1: DFS')
print('2: BFS')
print('3: A*')
choice = input('Enter your choice (1, 2, 3): ')

path = None
algo_text = ''
# Run the chosen algorithm
if choice == '1':
    print('Running DFS...')
    path = dfs(m, start, end)
    algo_text = 'DFS'
elif choice == '2':
    print('Running BFS...')
    path = bfs(m, start, end)
    algo_text = 'BFS'
elif choice == '3':
    print('Running A*...')
    path = a_star(m, start, end)
    algo_text = 'A*'
else:
    print('Invalid choice.')

# If a path is found, trace it with an agent
if path:
    a = agent(m, x=start[0], y=start[1], footprints=True)
    m.tracePath({a:path})
    l = textLabel(m, f'{algo_text} Path', f'{algo_text} Algorithm')
    m.run()
else:
    print(f'No path found from start to end using {algo_text}.')



Choose an algorithm:
1: DFS
2: BFS
3: A*
Enter your choice (1, 2, 3): 2
Running BFS...


In [3]:
from pyamaze import maze, agent, textLabel
from queue import PriorityQueue
import time

class aStar:
    def calNodeCost(self, node1, node2):
        nodex1, nodey1 = node1
        nodex2, nodey2 = node2
        return abs(nodex1 - nodex2) + abs(nodey1 - nodey2)
    
    def aStarInit(self, mazeDef, xtarget, ytarget):
        startNode = (mazeDef.rows, mazeDef.cols)
        nodeDist = {cell: float('inf') for cell in mazeDef.grid}
        totalNodeCost = {cell: float('inf') for cell in mazeDef.grid}
        return startNode, nodeDist, totalNodeCost 

    def aStar(self, mazeDef, xtarget, ytarget):
        start = time.time()
        startNode, nodeDist, totalNodeCost = self.aStarInit(mazeDef, xtarget, ytarget)
        nodeDist[startNode] = 0
        totalNodeCost[startNode] = self.calNodeCost(startNode, (xtarget, ytarget))
        algoPath = {}
        container = PriorityQueue()
        container.put((totalNodeCost[startNode], startNode))

       while not container.empty():
            _, currentNode = container.get()
            if currentNode == (xtarget, ytarget):
                break

            for direction in 'ESNW':
                if mazeDef.maze_map[currentNode][direction]:
                    if direction == 'E':
                        nextNode = (currentNode[0], currentNode[1] + 1)
                    elif direction == 'W':
                        nextNode = (currentNode[0], currentNode[1] - 1)
                    elif direction == 'S':
                        nextNode = (currentNode[0] + 1, currentNode[1])
                    elif direction == 'N':
                        nextNode = (currentNode[0] - 1, currentNode[1])

        tracePath = {}
        cell = (xtarget, ytarget)
        while cell != startNode:
            tracePath[algoPath[cell]] = cell
            cell = algoPath[cell]
        end = time.time()
        return tracePath, (end - start)

# Create a maze with pyamaze
m = maze(20, 20)
m.CreateMaze()

a_star_solver = aStar()
a_star_path, _ = a_star_solver.aStar(m, 1, 1)

# Visualize the path in pyamaze
a = agent(m, footprints=True)
m.tracePath({a: a_star_path})
m.run()


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 26)

In [ ]:
from pyamaze import maze, agent, textLabel
from queue import PriorityQueue

class Node:
    def __init__(self, x, y, parent=None):
        self.x = x
        self.y = y
        self.parent = parent
        self.g = 0  # Cost from start to node
        self.h = 0  # Heuristic cost from node to goal
        self.f = 0  # Total cost

    def __eq__(self, other):
        return self.x == other.x and self.y == other.y

    def __lt__(self, other):
        return self.f < other.f

def heuristic(node, goal):
    # Manhattan distance
    return abs(node.x - goal.x) + abs(node.y - goal.y)

def a_star(m, start, goal):
    start_node = Node(start[0], start[1])
    goal_node = Node(goal[0], goal[1])

    open_list = PriorityQueue()
    open_list.put((0, start_node))
    start_node.g = start_node.h = start_node.f = 0

    while not open_list.empty():
        _, current_node = open_list.get()

        if current_node == goal_node:
            path = {}
            while current_node.parent is not None:
                path[(current_node.x, current_node.y)] = (current_node.parent.x, current_node.parent.y)
                current_node = current_node.parent
            return path

        for direction in 'ESNW':
            if m.maze_map[current_node.x, current_node.y][direction]:
                if direction == 'E':
                    child = Node(current_node.x, current_node.y + 1, current_node)
                elif direction == 'W':
                    child = Node(current_node.x, current_node.y - 1, current_node)
                elif direction == 'S':
                    child = Node(current_node.x + 1, current_node.y, current_node)
                elif direction == 'N':
                    child = Node(current_node.x - 1, current_node.y, current_node)

                child.g = current_node.g + 1
                child.h = heuristic(child, goal_node)
                child.f = child.g + child.h

                for _, open_node in open_list.queue:
                    if child == open_node and child.g > open_node.g:
                        break
                else:
                    open_list.put((child.f, child))

    return None



def main():
    # Create the maze
    m = maze(20, 20)
    m.CreateMaze()

    # Define start and end points
    start = (1, 1)
    end = (m.rows, m.cols)

    # Find path using A*
    path = a_star(m, start, end)

    # Display the maze and path
    if path:
        a = agent(m, x=start[0], y=start[1], footprints=True)
        m.tracePath({a: path})
    else:
        print("No path found using A*")

    m.run()

if __name__ == "__main__":
    main()
